# Interfaces: From Protocols to ABCs


## Interfaces and Protocols in Python Culture

Even without an interface keyword in the language, and regardless of ABCs, every class has `an interface: the set public attributes (methods or data attributes) implemented or inherited by the class`. This includes special methods, like `__getitem__` or `__add__`.

A useful complementary definition of interface is: `the subset of an object’s public methods that enable it to play a specific role in the system.`

Protocols are interfaces, but because they are `informal—define`d only by documentation and conventions—protocols `cannot be enforced` like formal interfaces can (like ABCs)

## Python Digs Sequences


In [2]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
f = Foo()

In [3]:
f[1]

10

In [4]:
for i in f:
    print(i)

0
10
20


There is no method `__iter__` yet Foo instances are iterable because—as a fallback—when Python sees a `__getitem__` method, it tries to iterate over the object by calling that method with integer indexes starting with 0. Because Python is smart enough to iterate over Foo instances, it can also make the in operator work even if Foo has no `__contains__` method: it does a full scan to check if an item is present.

## Monkey-Patching to Implement a Protocol at Runtime

```py
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
```

1. Create a function that takes dec, position, and card as arguments
2. Assign that function to an attribute named `__setitem__`
3. deck can now be sorted because FrenchDeck now implements the necessary method of the mutable sequence protocol


This is an example of `monkey patching: changing a class or module at runtime, without touching the source cod`e. Monkey patching is powerful, but the code that does the actual patching is very tightly coupled with the program to be patched, often handling private and undocumented parts.

## Subclassing an ABC


In [6]:
import collections

Card =  collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
        for rank in self.ranks]
        
    
    def __delitem__(self, position): # 
        del self._cards[position]
        
    def insert(self, position, value): # 
        self._cards.insert(position, value)

1. subclassing MutableSequence forces us to implement `__delitem__`, an abstract method of that ABC.
2. We are also required to implement insert, the third abstract method of MutableSequence.


## ABCs in the Standard Library

### ABCs in collections.abc

- Iterable, Container, and Sized
  - s. Iterable supports iteration with `__iter__`, Container supports the in operator with `__contains__`, and Sized supports len() with `__len__`.
- Sequence, Mapping, and Set
  - These are the main immutable collection types, and each has a mutable subclass.
- MappingView
  - In Python 3, the objects returned from the mapping methods .items(), .keys(), and .values() inherit from ItemsView, ValuesView, and ValuesView, respectively.
- Callable and Hashable
   - Their main use is to support the insinstance built-in as a safe way of determining whether an object is callable or hashable.
- Iterator
  - Note that iterator subclasses Iterable

### The Numbers Tower of ABCs

- Number
- Complex
- Real
- Rational
- Integral

## Defining and Using an ABC


In [7]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""
        
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        
        This method should raise `LookupError` when the instance is empty.
         """

1. To define an ABC, subclass abc.ABC
2. An abstract method is marked with the @abstractmethod decorator, and often its body is empty except for a docstring.
3. An ABC may include concrete methods.

### Subclassing the Tombola ABC


In [8]:
class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom() 
        self._items = []
        self.load(items) 

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items) 
 
    def pick(self): 
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

1. This BingoCage class explicitly extends Tombola.
2. Explicitly implement the abstract methods with concrete ones

### A Virtual Subclass of Tombola

This is done by calling a register method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will be recognized as such by functions like issubclass and isinstance, but it will not inherit any methods or attributes from the ABC.

In [9]:
from random import randrange

@Tombola.register
class TomboList(list):
    
     def pick(self):
        if self: # 
            position = randrange(len(self))
            return self.pop(position) # 
        else:
            raise LookupError('pop from empty TomboList')

## Geese Can Behave as Ducks

A class can be recognized as a virtual subclass of an ABC even without registration

In [10]:
class Struggle:
    def __len__(self): return 23 

In [12]:
from collections import abc

print(isinstance(Struggle(), abc.Sized))
print(issubclass(Struggle, abc.Sized))

True
True


# Chapter Summary

1. Learned about Python protocols and that duck typing means that python cares only about the methods an object implements not necessarily the specific type
2. Learn about abstract classes as an alternative to Protocols